# Setting up a mock DB to store Songs, users, and playlists

![](https://github.com/spotify-song/DS/blob/master/api/data/DB_tables_illu.png?raw=true)

In [1]:
import sys
sys.path.insert(0, '../')
from spotify_users import User
from my_db import UpdateTables

In [2]:
user1 = User()
user1.user_top_50('')



('Agustinvargas',) not in db
('Agustinvargas',) was added to table


{'Display Name': 'Agustinvargas',
 'Tokens Info': 'token_info',
 'User ID': 'avargas-274',
 'Top Track IDs': ['0akyEssGRVHstqCSWXusJL',
  '02gaYAEdeR6poHcBH1KUQF',
  '0NeJjNlprGfZpeX2LQuN6c',
  '6plO0gM4tUvRC9TKFGIuaN',
  '54KsfVVnN4YWI2mMrnyUcC',
  '5iSpfk6cDOSYePagAoG639',
  '1jecO8NeYLsVWVptITz4c1',
  '1s3WD4gbNoEXHiuSTmAKaK',
  '6KseaEAFSS63N2NPZtDnRL',
  '57mLRN6tfXwTRvp9oPWpop',
  '52N0IV8hLVkRmnpFclmCzK',
  '4zFPUEMucYleIIUnYVoeZf',
  '7kWFRZdedr2gtfE8JDumVZ',
  '1IF5UcqRO42D12vYwceOY6',
  '2UxrK7r4cyQOSh7wvdQTe1',
  '0107Auhv91hE49iLoxtayt',
  '4cJOLN346rtOty3UPACsao',
  '17OkYffr0SdAcpcbwMkDDG',
  '71Mj2THXRicZhTFGzln3al',
  '0JfsIu62NVXNQl2s7ATN37',
  '6Zy0ITa16EjCAbbGuPzdRi',
  '1sJev5Y7VI2Ke8AwUpnh0l',
  '4RNYL9drYkmWYpDyfknta9',
  '7v3YlquaNhK2GYKzxovSEp',
  '4RrOSjdnV8rkpIuOIfkKYS',
  '1yTJg3lyUPmwbnve82twH5',
  '3PUbNbybe6dTMWdUt9vQ02',
  '51wUFdgpNsV8cVzu7i6N0l',
  '6fcS6fncRVP8rldHjriZHS',
  '0xSqHQ5wv80hNkpU50vPc7',
  '2TzkIzgzIHhewMxyh1u4hh',
  '5ONAA8z6SvFBniu8zXz1A

## Adding user's `display_name` to the users table

In [3]:
from os import getenv
from dotenv import load_dotenv
import psycopg2

load_dotenv()

# connecting to DB
conn = psycopg2.connect(user=getenv('DB_USER'),
                        password=getenv('DB_PASSWORD'),
                        host=getenv('DB_HOST'),
                        port=getenv('DB_PORT'),
                        database=getenv('DB_DATABASE'))

curs = conn.cursor()

# execute to users table
user_id = ''
display_name = 'Agustinvargas'

user_tbl_insert_query = """
    INSERT INTO users (DISPLAY_NAME)
    VALUES (%s)
    """
check_for_user_query = """
    SELECT display_name
    FROM users
    WHERE display_name = (%s)
    """
check_for_user = (user_id,)

record_to_insert = (user_id,)

# checks for user in DB
user_check = curs.execute(check_for_user_query, check_for_user)

# if user exists, close connection
if curs.fetchone() == check_for_user:
    print(f"{user_id} exists in db")

# If user does not exist, insert into db
else:
    print("failed to find user in table")
    curs.execute(user_tbl_insert_query, record_to_insert)
    curs.execute(check_for_user_query, check_for_user)
    record = curs.fetchone()
    conn.commit()
    print(f"{record} was added to table")

curs.close()
conn.close()
print("PostgreSQL connection is closed")


('cody',) already in DB
